In [2]:
import pandas as pd
import folium
from folium.plugins import HeatMap

In [5]:
!pip install pyarrow

In [18]:
%%time
mybicard = pd.read_parquet('mybicard.parquet', engine='pyarrow', columns=['transdate', 'stop_id', 'route_nm', 'normalcnt','studentcnt','childcnt','transflag'])

Wall time: 53.7 s


In [19]:
mybicard_401 = mybicard.loc[(mybicard["route_nm"] == "401")].reset_index(drop=True)

In [20]:
# 전송일자 데이트 포맷으로 변환
mybicard_401["transdate"] = pd.to_datetime(mybicard_401["transdate"], format = "%Y%m%d %H:%M:%S")

In [21]:
# 전체 승객 수 변수 생성(일반 + 학생 + 아동)
mybicard_401["totalcnt"] = mybicard_401[["normalcnt", "studentcnt", "childcnt"]].sum(axis = 1)

In [39]:
# 21948
mybicard_401_agg_10m = (mybicard_401.loc[(mybicard_401["transflag"] != "하차")]
                                   .groupby(["route_nm", "stop_id", 
                                             pd.Grouper(key="transdate", freq='30Min')])
                                   .agg({"totalcnt" : sum, "normalcnt":sum, "studentcnt":sum, "childcnt":sum})
                                   .reset_index())

In [40]:
mybicard_401_agg_10m.head(25)

,route_nm,stop_id,transdate,totalcnt,normalcnt,studentcnt,childcnt
0,401,3100020,2020-04-08 06:30:00,2,2,0,0
1,401,3100020,2020-04-08 08:00:00,3,3,0,0
2,401,3100020,2020-04-08 08:30:00,4,4,0,0
3,401,3100020,2020-04-08 10:00:00,2,2,0,0
4,401,3100020,2020-04-08 11:00:00,2,2,0,0
5,401,3100020,2020-04-08 12:00:00,1,1,0,0
6,401,3100020,2020-04-08 13:00:00,2,2,0,0
7,401,3100020,2020-04-08 13:30:00,1,0,1,0
8,401,3100020,2020-04-08 14:00:00,1,0,1,0
9,401,3100020,2020-04-08 15:30:00,4,4,0,0


In [49]:
mybicard_401_agg_30m = mybicard_401_agg_10m[mybicard_401_agg_10m['transdate'].dt.hour == 8].copy().reset_index()

In [51]:
mybicard_401_agg_30m.head(10)

,index,route_nm,stop_id,transdate,totalcnt,normalcnt,studentcnt,childcnt
0,1,401,3100020,2020-04-08 08:00:00,3,3,0,0
1,2,401,3100020,2020-04-08 08:30:00,4,4,0,0
2,22,401,3100020,2020-04-09 08:00:00,3,3,0,0
3,23,401,3100020,2020-04-09 08:30:00,1,1,0,0
4,36,401,3100020,2020-04-10 08:00:00,3,3,0,0
5,37,401,3100020,2020-04-10 08:30:00,3,3,0,0
6,57,401,3100020,2020-04-11 08:00:00,1,1,0,0
7,93,401,3100020,2020-04-13 08:30:00,3,3,0,0
8,116,401,3100020,2020-04-14 08:00:00,3,3,0,0
9,117,401,3100020,2020-04-14 08:30:00,6,6,0,0


In [32]:
import datetime

In [42]:
mybicard_401_agg_10m[(mybicard_401_agg_10m['transdate'] == mybicard_401_agg_10m.loc[num]['transdate'] + datetime.timedelta(days=1)) &
                     (mybicard_401_agg_10m['stop_id'] == mybicard_401_agg_10m.loc[num]['stop_id'])]

,route_nm,stop_id,transdate,totalcnt,normalcnt,studentcnt,childcnt
23,401,3100020,2020-04-09 08:30:00,1,1,0,0


In [80]:
num=2

In [76]:
mybicard_401_agg_10m[(mybicard_401_agg_10m['transdate'] == mybicard_401_agg_10m.loc[num]['transdate'] + datetime.timedelta(days=1)) &
                     (mybicard_401_agg_10m['stop_id'] == mybicard_401_agg_10m.loc[num]['stop_id'])][['totalcnt','normalcnt','studentcnt','childcnt']]

,totalcnt,normalcnt,studentcnt,childcnt


In [100]:
a.shape

(100, 7)

In [262]:
a = mybicard_401_agg_10m[:100].copy()

In [155]:
a[24:26]

,route_nm,stop_id,transdate,totalcnt,normalcnt,studentcnt,childcnt
24,401,3100020,2020-04-09 09:00:00,1,1,0,0
25,401,3100020,2020-04-09 09:30:00,3,3,0,0


In [184]:
def period_before_cnt(df):
    result_df = df.sort_values(['stop_id', 'transdate']).copy().reset_index()
    col_names = ['totalcnt','normalcnt','studentcnt','childcnt']
    last_stop_id = ''
    for i in range(len(result_df)):
        if last_stop_id != result_df.loc[i]['stop_id'] :
            subset_df = result_df[result_df['stop_id'] == result_df.loc[i]['stop_id']].copy()
            last_stop_id = result_df.loc[i]['stop_id']
        temp_df = subset_df[(subset_df['transdate'] == result_df.loc[i]['transdate'] - datetime.timedelta(days=1))][col_names].copy()
        j = 0
        while len(temp_df) == 0:
            j += 1
            if subset_df['transdate'].min() >= data['transdate'] - datetime.timedelta(days=j):
                temp_df = data[col_names].copy()
                break
            temp_df = subset_df[(subset_df['transdate'] == data['transdate'] - datetime.timedelta(days=j))][col_names].copy()
    return temp_df

In [251]:
def period_before_cnt(data, df):
    result_df = df.sort_values(['stop_id', 'transdate']).copy().reset_index()
    col_names = ['totalcnt','normalcnt','studentcnt','childcnt']
    subset_df = result_df[result_df['stop_id'] == data['stop_id']].copy()
    temp_df = subset_df[(subset_df['transdate'] == data['transdate'] - datetime.timedelta(days=1))][col_names].copy()
    j = 0
    while len(temp_df) == 0:
        j += 1
        if subset_df['transdate'].min() >= data['transdate'] - datetime.timedelta(days=j):
            temp_df = subset_df[(subset_df['transdate'] == data['transdate'])][col_names].copy()
            break
        temp_df = subset_df[(subset_df['transdate'] == data['transdate'] - datetime.timedelta(days=j))][col_names].copy()
    return temp_df.reset_index(drop=True).loc[0]

In [257]:
def period_before_cnt(data, df):
    result_df = df.sort_values(['stop_id', 'transdate']).copy().reset_index()
    col_names = ['totalcnt','normalcnt','studentcnt','childcnt']
    subset_df = result_df[result_df['stop_id'] == data['stop_id']].copy()
    temp_df = subset_df[(subset_df['transdate'] == data['transdate'] - datetime.timedelta(days=1))][col_names].copy()
    j = 0
    while len(temp_df) == 0:
        j += 1
        if subset_df['transdate'].min() >= data['transdate'] - datetime.timedelta(days=j):
            temp_df = subset_df[(subset_df['transdate'] == data['transdate'])][col_names].copy()
            break
        temp_df = subset_df[(subset_df['transdate'] == data['transdate'] - datetime.timedelta(days=j))][col_names].copy()
    return temp_df.reset_index(drop=True).loc[0]

In [259]:
from functools import partial

In [264]:
make_columns = partial(period_before_cnt, df= a)

In [263]:
a.head(3)

,route_nm,stop_id,transdate,totalcnt,normalcnt,studentcnt,childcnt
0,401,3100020,2020-04-08 06:30:00,2,2,0,0
1,401,3100020,2020-04-08 08:00:00,3,3,0,0
2,401,3100020,2020-04-08 08:30:00,4,4,0,0


In [284]:
def period_before_cnt(data, df):
    result_df = df.sort_values(['stop_id', 'transdate']).copy().reset_index()
    col_names = ['totalcnt','normalcnt','studentcnt','childcnt']
    subset_df = result_df[result_df['stop_id'] == data['stop_id']].copy()
    temp_df = subset_df[(subset_df['transdate'] == data['transdate'] - datetime.timedelta(days=1))][col_names].copy()
    j = 0
    while len(temp_df) == 0:
        j += 1
        if subset_df['transdate'].min() >= data['transdate'] - datetime.timedelta(days=j):
            temp_df = subset_df[(subset_df['transdate'] == data['transdate'])][col_names].copy()
            break
        temp_df = subset_df[(subset_df['transdate'] == data['transdate'] - datetime.timedelta(days=j))][col_names].copy()
    return temp_df.reset_index(drop=True).loc[0]

In [285]:
make_columns = partial(period_before_cnt, df= a)

In [ ]:
 

출처: https://curryyou.tistory.com/179 [카레유]

In [295]:
!pip --trusted-host pypi.org --trusted-host files.pythonhosted.org install tqdm

In [296]:
from tqdm import tqdm

In [297]:
tqdm.pandas()

In [298]:
a[new_col_names] = a.progress_apply(lambda x: (make_columns(x)), axis=1)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 140.45it/s]


In [288]:
a.head(30)

,route_nm,stop_id,transdate,totalcnt,normalcnt,studentcnt,childcnt,totalcnt_D-1,normalcnt_D-1,studentcnt_D-1,childcnt_D-1
0,401,3100020,2020-04-08 06:30:00,2,2,0,0,2,2,0,0
1,401,3100020,2020-04-08 08:00:00,3,3,0,0,3,3,0,0
2,401,3100020,2020-04-08 08:30:00,4,4,0,0,4,4,0,0
3,401,3100020,2020-04-08 10:00:00,2,2,0,0,2,2,0,0
4,401,3100020,2020-04-08 11:00:00,2,2,0,0,2,2,0,0
5,401,3100020,2020-04-08 12:00:00,1,1,0,0,1,1,0,0
6,401,3100020,2020-04-08 13:00:00,2,2,0,0,2,2,0,0
7,401,3100020,2020-04-08 13:30:00,1,0,1,0,1,0,1,0
8,401,3100020,2020-04-08 14:00:00,1,0,1,0,1,0,1,0
9,401,3100020,2020-04-08 15:30:00,4,4,0,0,4,4,0,0


In [253]:
list(b)

['totalcnt', 'normalcnt', 'studentcnt', 'childcnt']

In [236]:
to_list(b[0])

NameError: name 'to_list' is not defined

In [221]:
type(b)

pandas.core.frame.DataFrame

In [227]:
b.loc[25]

totalcnt      3
normalcnt     3
studentcnt    0
childcnt      0
Name: 25, dtype: int64

In [205]:
a.columns

Index(['route_nm', 'stop_id', 'transdate', 'totalcnt', 'normalcnt',
       'studentcnt', 'childcnt', 'totalcnt_D-1', 'normalcnt_D-1',
       'studentcnt_D-1', 'childcnt_D-1'],
      dtype='object')

In [210]:
a[new_col_names][26] = period_before_cnt(a.loc[26], a)

ValueError: Wrong number of items passed 4, placement implies 1

In [209]:
a['totalcnt_D-1'][24]

nan

In [191]:
aa = period_before_cnt(a.loc[26], a)

1


In [193]:
aa.index

Int64Index([5], dtype='int64')

In [112]:
df = a.copy()

In [113]:
result_df = df.sort_values(['stop_id', 'transdate']).copy().reset_index()
col_names = ['totalcnt','normalcnt','studentcnt','childcnt']
last_stop_id = ''

In [ ]:
import tqdm

In [119]:
for i in range(len(result_df)):
    if last_stop_id != result_df.loc[i]['stop_id'] :
        subset_df = result_df[result_df['stop_id'] == result_df.loc[i]['stop_id']].copy()
        last_stop_id = result_df.loc[i]['stop_id']
    temp_df = subset_df[(subset_df['transdate'] == result_df.loc[i]['transdate'] - datetime.timedelta(days=1))][col_names].copy()
    j = 1
    while len(temp_df) != 0:
        j += 1
        if subset_df['transdate'].min() >= result_df.loc[i]['transdate'] - datetime.timedelta(days=j):
            temp_df = result_df.loc[i][col_names]
            break
    new_col_names = [col_name + '_D-1' for col_name in col_names]

In [133]:
result_df[col].loc[99] = list(temp_df)[i]

KeyError: 'totalcnt_D-1'

In [132]:
result_df

,index,route_nm,stop_id,transdate,totalcnt,normalcnt,studentcnt,childcnt
0,0,401,3100020,2020-04-08 06:30:00,2,2,0,0
1,1,401,3100020,2020-04-08 08:00:00,3,3,0,0
2,2,401,3100020,2020-04-08 08:30:00,4,4,0,0
3,3,401,3100020,2020-04-08 10:00:00,2,2,0,0
4,4,401,3100020,2020-04-08 11:00:00,2,2,0,0
...,...,...,...,...,...,...,...,...
95,95,401,3100020,2020-04-13 09:30:00,1,1,0,0
96,96,401,3100020,2020-04-13 10:00:00,1,1,0,0
97,97,401,3100020,2020-04-13 10:30:00,1,1,0,0
98,98,401,3100020,2020-04-13 11:00:00,2,2,0,0


In [124]:
list(temp_df)

[4, 3, 1, 0]

In [128]:
new_col_names

['totalcnt_D-1', 'normalcnt_D-1', 'studentcnt_D-1', 'childcnt_D-1']

In [130]:
result_df[[new_col_names]].loc[99] = list(temp_df)

KeyError: "None of [Index([('totalcnt_D-1', 'normalcnt_D-1', 'studentcnt_D-1', 'childcnt_D-1')], dtype='object')] are in the [columns]"

In [110]:
b = period_before_cnt(a)

Empty DataFrame
Columns: [totalcnt, normalcnt, studentcnt, childcnt]
Index: []


<ipython-input-109-77ff9b71956b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.loc[i][new_col_names] = temp_df


KeyError: "None of [Index(['totalcnt_D-1', 'normalcnt_D-1', 'studentcnt_D-1', 'childcnt_D-1'], dtype='object')] are in the [index]"